<a href="https://colab.research.google.com/github/woo13sd/rupture-theory/blob/main/notebooks/Rupture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Install all required packages and create project directories

# System packages
!apt-get update -qq
!apt-get install -y gfortran wget unzip

# Python packages
!pip install numpy matplotlib scipy h5py cython --quiet

# Create only the necessary directories
!mkdir -p /content/data
!mkdir -p /content/images
!mkdir -p /content/camb_utils

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gfortran is already the newest version (4:11.2.0-1ubuntu1).
unzip is already the newest version (6.0-26ubuntu3.2).
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.


In [2]:
# Cell 2: Clone CAMB from GitHub (main branch)
!git clone --depth=1 https://github.com/cmbant/CAMB.git /content/CAMB

# List to confirm
!ls /content/CAMB

Cloning into '/content/CAMB'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 147 (delta 8), reused 63 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (147/147), 13.35 MiB | 7.29 MiB/s, done.
Resolving deltas: 100% (8/8), done.
camb		 fortran   LICENCE.txt	   README.rst
CONTRIBUTING.md  forutils  MANIFEST.in	   requirements.txt
docs		 inifiles  pyproject.toml  setup.py


In [4]:
# Cell 3: Copy your uploaded custom .f90 files into the CAMB fortran directory (correct case)
import shutil
import os

source_dir = "/content/camb_utils"
target_dir = "/content/CAMB/fortran"

# Updated file names with correct casing
f90_files = [
    "InitialPower.f90",
    "reionization.f90"
]

for file in f90_files:
    src = os.path.join(source_dir, file)
    dst = os.path.join(target_dir, file)
    shutil.copyfile(src, dst)

print("✅ Modified .f90 files copied to CAMB/fortran/")

✅ Modified .f90 files copied to CAMB/fortran/


In [5]:
# Cell 4: Copy custom .ini files to CAMB/inifiles/
import shutil
import os

ini_files = [
    "rt_f1e08.ini",
    "rt_f1e05.ini",
    "rt_f1e02.ini"
]

source_dir = "/content/camb_utils"
target_dir = "/content/CAMB/inifiles"

for file in ini_files:
    src = os.path.join(source_dir, file)
    dst = os.path.join(target_dir, file)
    shutil.copyfile(src, dst)

print("✅ Custom .ini files copied to CAMB/inifiles/")

✅ Custom .ini files copied to CAMB/inifiles/


In [6]:
# Cell 5a: Compile constants.f90 manually
import subprocess

build_dir = "/content/CAMB/fortran"
filename = "constants.f90"

# Compile
compile_cmd = [
    "gfortran",
    "-c",                               # Compile only (no linking)
    "-J", build_dir,                    # Output .mod files here
    "-o", f"{build_dir}/constants.o",  # Output .o file
    f"{build_dir}/{filename}",         # Input source file
]

# Run command
result = subprocess.run(compile_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Show result
if result.returncode == 0:
    print(f"✅ {filename} compiled successfully.")
else:
    print(f"❌ Error compiling {filename}:\n{result.stderr}")

✅ constants.f90 compiled successfully.


In [7]:
# Cell 6: Compile MiscUtils.f90 with proper lowercase handling for the .mod file

import subprocess
import shutil
import os

miscutils_src = "/content/forutils/MiscUtils.f90"
mod_file = "miscutils.mod"
obj_file = "MiscUtils.o"

print("🔧 Cell 6: Compiling MiscUtils.f90...")

try:
    subprocess.run(["gfortran", "-c", miscutils_src], check=True)

    if os.path.exists(obj_file) and os.path.exists(mod_file):
        shutil.move(obj_file, "/content/CAMB/fortran/")
        shutil.move(mod_file, "/content/CAMB/fortran/")
        print("✅ Cell 6: Compilation successful. Files moved.")
    else:
        print("❌ Cell 6: Compilation did not produce expected output files.")
except subprocess.CalledProcessError as e:
    print("❌ Cell 6: Compilation error.")
    print(e)

🔧 Cell 6: Compiling MiscUtils.f90...
❌ Cell 6: Compilation error.
Command '['gfortran', '-c', '/content/forutils/MiscUtils.f90']' returned non-zero exit status 1.


In [8]:
# Cell 7: Download forutils and verify FileUtils.f90 is available

import os
import subprocess

print("🌐 Cloning CAMB forutils repo...")

# Clean up if already exists (optional, comment out if not needed)
if os.path.exists("/content/CAMB/forutils"):
    subprocess.run(["rm", "-rf", "/content/CAMB/forutils"])

# Clone the repository
subprocess.run(["git", "clone", "https://github.com/cmbant/forutils.git", "/content/CAMB/forutils"])

# Check required file
required = "FileUtils.f90"
file_path = f"/content/CAMB/forutils/{required}"

if os.path.exists(file_path):
    print(f"✅ {required} found at {file_path}")
else:
    raise FileNotFoundError(f"❌ {required} not found in /content/CAMB/forutils/")

🌐 Cloning CAMB forutils repo...
✅ FileUtils.f90 found at /content/CAMB/forutils/FileUtils.f90


In [17]:
# 🛠 Cell 8: Patch config.f90 to remove MpiUtils reference

config_path = "/content/CAMB/fortran/config.f90"

with open(config_path, "r") as f:
    lines = f.readlines()

patched_lines = []
for line in lines:
    if "use MpiUtils" in line:
        patched_lines.append("! use MpiUtils  ! 🔧 MPI disabled in Colab\n")
    else:
        patched_lines.append(line)

with open(config_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 8: Patched config.f90 to bypass MPI dependency.")

✅ Cell 8: Patched config.f90 to bypass MPI dependency.


In [22]:
# 🛠️ Cell 9: Patch FileUtils.f90 to disable MPI dependency

fileutils_path = "/content/CAMB/forutils/FileUtils.f90"

with open(fileutils_path, "r") as f:
    lines = f.readlines()

patched_lines = []
for line in lines:
    if "use MpiUtils" in line:
        patched_lines.append("! use MpiUtils  ! 🔧 MPI disabled for Colab\n")
    else:
        patched_lines.append(line)

with open(fileutils_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 9: Patched FileUtils.f90 to bypass MPI dependency.")

✅ Cell 9: Patched FileUtils.f90 to bypass MPI dependency.


In [29]:
# 🔧 Cell 11: Recompile MiscUtils.f90 with corrected path

import subprocess
import shutil
import os

miscutils_src = "/content/CAMB/forutils/MiscUtils.f90"
mod_file = "miscutils.mod"
obj_file = "MiscUtils.o"

print("🔧 Cell 11: Compiling MiscUtils.f90...")

try:
    subprocess.run(["gfortran", "-c", miscutils_src], check=True)

    if os.path.exists(obj_file) and os.path.exists(mod_file):
        shutil.move(obj_file, "/content/CAMB/fortran/")
        shutil.move(mod_file, "/content/CAMB/fortran/")
        print("✅ Cell 11: Compilation successful. Files moved.")
    else:
        print("❌ Cell 11: Compilation did not produce expected output files.")
except subprocess.CalledProcessError as e:
    print("❌ Cell 11: Compilation error.")
    print(e)

🔧 Cell 11: Compiling MiscUtils.f90...
✅ Cell 11: Compilation successful. Files moved.


In [34]:
# 🛠 Cell 14: Patch StringUtils.f90 to bypass MPI dependency

string_utils_path = "/content/CAMB/forutils/StringUtils.f90"

with open(string_utils_path, "r") as f:
    lines = f.readlines()

patched_lines = []
for line in lines:
    if "use MpiUtils" in line:
        patched_lines.append("! use MpiUtils  ! 🔧 MPI disabled in Colab\n")
    else:
        patched_lines.append(line)

with open(string_utils_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 14: Patched StringUtils.f90 to bypass MPI dependency.")

✅ Cell 14: Patched StringUtils.f90 to bypass MPI dependency.


In [35]:
# 🔧 Cell 15: Compiling StringUtils.f90 after MPI patch

import subprocess

src = "/content/CAMB/forutils/StringUtils.f90"

print("🔧 Cell 15: Compiling StringUtils.f90...")

try:
    subprocess.run(["gfortran", "-I", "/content/CAMB/fortran", "-c", src], check=True)
    print("✅ Cell 15: Compilation successful.")
except subprocess.CalledProcessError as e:
    print("❌ Cell 15: Compilation failed.")
    print(e)

🔧 Cell 15: Compiling StringUtils.f90...
✅ Cell 15: Compilation successful.


In [36]:
# 🔧 Cell 16: Recompiling FileUtils.f90 after StringUtils resolved

import subprocess

src = "/content/CAMB/forutils/FileUtils.f90"

print("🔧 Cell 16: Compiling FileUtils.f90...")

try:
    subprocess.run(["gfortran", "-I", "/content/CAMB/fortran", "-c", src], check=True)
    print("✅ Cell 16: Compilation successful.")
except subprocess.CalledProcessError as e:
    print("❌ Cell 16: Compilation failed.")
    print(e)

🔧 Cell 16: Compiling FileUtils.f90...
✅ Cell 16: Compilation successful.


In [37]:
# 🔧 Cell 17: Recompile config.f90 now that dependencies are resolved

import subprocess

src = "/content/CAMB/fortran/config.f90"

print("🔧 Cell 17: Compiling config.f90...")

try:
    subprocess.run(["gfortran", "-I", "/content/CAMB/fortran", "-c", src], check=True)
    print("✅ Cell 17: Compilation successful.")
except subprocess.CalledProcessError as e:
    print("❌ Cell 17: Compilation failed.")
    print(e)

🔧 Cell 17: Compiling config.f90...
✅ Cell 17: Compilation successful.


In [44]:
# 🔧 Cell 21: Patch RangeUtils.f90 to bypass MPI dependency

rangeutils_path = "/content/CAMB/forutils/RangeUtils.f90"

with open(rangeutils_path, "r") as f:
    lines = f.readlines()

patched_lines = []
for line in lines:
    if "use MpiUtils" in line:
        patched_lines.append("! use MpiUtils  ! 🔧 MPI disabled in Colab\n")
    else:
        patched_lines.append(line)

with open(rangeutils_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 21: Patched RangeUtils.f90 to bypass MPI dependency.")

✅ Cell 21: Patched RangeUtils.f90 to bypass MPI dependency.


In [48]:
# 🛠 Cell 24: Patch ArrayUtils.f90 to bypass MPI

arrayutils_path = "/content/CAMB/forutils/ArrayUtils.f90"

with open(arrayutils_path, "r") as f:
    lines = f.readlines()

patched_lines = []
for line in lines:
    if "use MpiUtils" in line:
        patched_lines.append("! use MpiUtils  ! 🔧 MPI disabled in Colab\n")
    else:
        patched_lines.append(line)

with open(arrayutils_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 24: Patched ArrayUtils.f90 to bypass MPI dependency.")

✅ Cell 24: Patched ArrayUtils.f90 to bypass MPI dependency.


In [49]:
# 🔧 Cell 25: Compile ArrayUtils.f90

import subprocess

src_path = "/content/CAMB/forutils/ArrayUtils.f90"

try:
    subprocess.run(
        ["gfortran", "-I", "/content/CAMB/fortran", "-c", src_path],
        check=True
    )
    print("✅ Cell 25: Successfully compiled ArrayUtils.f90")
except subprocess.CalledProcessError as e:
    print("❌ Cell 25: Compilation failed.")
    print(e)

✅ Cell 25: Successfully compiled ArrayUtils.f90


In [52]:
# 🔧 Cell 27: Patch RangeUtils.f90 to remove preprocessor blocks

range_utils_path = "/content/CAMB/forutils/RangeUtils.f90"

with open(range_utils_path, "r") as f:
    lines = f.readlines()

patched_lines = []
skip_block = False

for line in lines:
    if "#ifdef" in line or "#ifndef" in line or "#else" in line or "#endif" in line:
        # Comment out preprocessor lines
        patched_lines.append(f"! {line}")
        continue
    patched_lines.append(line)

with open(range_utils_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 27: Patched RangeUtils.f90 to bypass preprocessor directives.")

✅ Cell 27: Patched RangeUtils.f90 to bypass preprocessor directives.


In [55]:
# 🔧 Cell 29: Patch invalid impure elemental subroutine in RangeUtils.f90

path = "/content/CAMB/forutils/RangeUtils.f90"

with open(path, "r") as f:
    lines = f.readlines()

patched = []
for line in lines:
    if "impure elemental subroutine TRanges_Free" in line:
        patched.append("subroutine TRanges_Free(this)  ! 🔧 patched to remove impure elemental\n")
    elif "#else" in line or "#endif" in line or "#ifdef" in line:
        continue  # Remove all illegal preprocessor directives
    else:
        patched.append(line)

with open(path, "w") as f:
    f.writelines(patched)

print("✅ Cell 29: Patched RangeUtils.f90 to remove impure elemental and preprocessor directives.")

✅ Cell 29: Patched RangeUtils.f90 to remove impure elemental and preprocessor directives.


In [58]:
# ✅ Cell 31: Final patch for RangeUtils.f90

rangeutils_path = "/content/CAMB/forutils/RangeUtils.f90"

with open(rangeutils_path, "r") as f:
    lines = f.readlines()

patched_lines = []
skip = False
for line in lines:
    if "#ifdef __GFORTRAN__" in line or "#else" in line or "#endif" in line:
        skip = True
        continue
    if skip and "subroutine TRanges_Free" in line:
        patched_lines.append(line.lstrip())  # remove leading spaces
        skip = False
    else:
        patched_lines.append(line)

with open(rangeutils_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 31: Cleaned preprocessor logic in RangeUtils.f90")

✅ Cell 31: Cleaned preprocessor logic in RangeUtils.f90


In [61]:
# 🔧 Cell 33: Fix duplicate subroutine definition in RangeUtils.f90

src_path = "/content/CAMB/forutils/RangeUtils.f90"

with open(src_path, "r") as f:
    lines = f.readlines()

fixed_lines = []
skip_next = False
for i, line in enumerate(lines):
    if "subroutine TRanges_Free(this)" in line and not skip_next:
        fixed_lines.append("  subroutine TRanges_Free(this)\n")
        skip_next = True  # Skip next identical definition
    elif "subroutine TRanges_Free(this)" in line and skip_next:
        continue  # Skip duplicate
    else:
        fixed_lines.append(line)

with open(src_path, "w") as f:
    f.writelines(fixed_lines)

print("✅ Cell 33: Duplicate subroutine definition removed from RangeUtils.f90")

✅ Cell 33: Duplicate subroutine definition removed from RangeUtils.f90


In [62]:
# 🔧 Cell 34: Compile RangeUtils.f90 after duplicate fix
import subprocess

try:
    subprocess.run(
        ["gfortran", "-I", "/content/CAMB/fortran", "-c", "/content/CAMB/forutils/RangeUtils.f90"],
        check=True
    )
    print("✅ Cell 34: Successfully compiled RangeUtils.f90")
except subprocess.CalledProcessError as e:
    print("❌ Cell 34: Compilation failed.")
    print(e)

✅ Cell 34: Successfully compiled RangeUtils.f90


In [70]:
# ✅ Cell 39: Patch MathUtils.f90 to bypass MPI dependency

file_path = "/content/CAMB/fortran/MathUtils.f90"

with open(file_path, "r") as f:
    lines = f.readlines()

patched_lines = []
for line in lines:
    if "use MpiUtils" in line:
        patched_lines.append("! " + line.strip() + "  ! 🔧 MPI disabled\n")
    else:
        patched_lines.append(line)

with open(file_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 39: Patched MathUtils.f90 to bypass MPI dependency.")

✅ Cell 39: Patched MathUtils.f90 to bypass MPI dependency.


In [71]:
# 🔧 Cell 40: Compile MathUtils.f90

import subprocess

src_path = "/content/CAMB/fortran/MathUtils.f90"

try:
    subprocess.run(["gfortran", "-I", "/content/CAMB/fortran", "-c", src_path], check=True)
    print("✅ Cell 40: Successfully compiled MathUtils.f90")
except subprocess.CalledProcessError as e:
    print("❌ Cell 40: Compilation failed.")
    print(e)

✅ Cell 40: Successfully compiled MathUtils.f90


In [78]:
# 🔧 Cell 44: Patch classes.f90 to bypass MPI usage

file_path = "/content/CAMB/fortran/classes.f90"

with open(file_path, "r") as f:
    lines = f.readlines()

patched_lines = []
for line in lines:
    if "use MpiUtils" in line:
        patched_lines.append("! use MpiUtils  ! 🔧 MPI disabled in Colab\n")
    else:
        patched_lines.append(line)

with open(file_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 44: Patched classes.f90 to bypass MPI dependency.")

✅ Cell 44: Patched classes.f90 to bypass MPI dependency.


In [80]:
# 🩹 Cell 46: Patch classes.f90 to bypass MPI dependency

classes_path = "/content/CAMB/fortran/classes.f90"

with open(classes_path, "r") as f:
    lines = f.readlines()

patched_lines = []
for line in lines:
    if "use MpiUtils" in line:
        patched_lines.append("! use MpiUtils  ! 🔧 MPI disabled in Colab\n")
    else:
        patched_lines.append(line)

with open(classes_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 46: Patched classes.f90 to bypass MPI dependency.")

✅ Cell 46: Patched classes.f90 to bypass MPI dependency.


In [91]:
# ✅ Cell 51: Patch Interpolation.f90 to bypass MPI dependency

src_path = "/content/CAMB/forutils/Interpolation.f90"

with open(src_path, "r") as f:
    lines = f.readlines()

patched_lines = []
for line in lines:
    if "use MpiUtils" in line:
        patched_lines.append("! use MpiUtils  ! 🔧 MPI disabled in Colab\n")
    else:
        patched_lines.append(line)

with open(src_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Patched Interpolation.f90 to bypass MPI.")

✅ Patched Interpolation.f90 to bypass MPI.


In [93]:
# 🩹 Cell 53: Patch Interpolation.f90 to bypass MPI dependency

interp_path = "/content/CAMB/forutils/Interpolation.f90"

with open(interp_path, "r") as f:
    lines = f.readlines()

patched_lines = []
for line in lines:
    if "use MpiUtils" in line:
        patched_lines.append("! use MpiUtils  ! 🔧 MPI disabled in Colab\n")
    else:
        patched_lines.append(line)

with open(interp_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 53: Patched Interpolation.f90 to bypass MPI dependency.")

✅ Cell 53: Patched Interpolation.f90 to bypass MPI dependency.


In [95]:
# 🔧 Cell 55: Patch Interpolation.f90 to disable MpiUtils usage

interpolation_path = "/content/CAMB/forutils/Interpolation.f90"

with open(interpolation_path, "r") as f:
    lines = f.readlines()

patched_lines = []
for line in lines:
    if "use MpiUtils" in line:
        patched_lines.append("! " + line)  # Comment out MPI
    else:
        patched_lines.append(line)

with open(interpolation_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 55: Patched Interpolation.f90 to bypass MPI dependency.")

✅ Cell 55: Patched Interpolation.f90 to bypass MPI dependency.


In [97]:
# 🔧 Cell 57: Patch and compile Interpolation.f90 to bypass MPI and fix preprocessor usage

import pathlib
import subprocess

src_path = pathlib.Path("/content/CAMB/forutils/Interpolation.f90")

# Read and patch the first lines of Interpolation.f90
patched_lines = []
with src_path.open("r") as f:
    lines = f.readlines()
    for line in lines:
        if "use MpiUtils" in line:
            patched_lines.append("!   use MpiUtils         ! 🔧 Patched: disable MPI dependency\n")
        elif "#ifdef SINGLE" in line:
            patched_lines.append("    integer, parameter :: dp=kind(1.0)\n")
        elif "#else" in line or "#endif" in line:
            continue  # Skip illegal preprocessor lines
        else:
            patched_lines.append(line)
        if "implicit none" in line:
            break  # Done patching once we've hit the main body

# Add the remaining lines unmodified
remaining = lines[len(patched_lines):]
patched_lines.extend(remaining)

# Write the patched file back
with src_path.open("w") as f:
    f.writelines(patched_lines)

# Now compile
try:
    subprocess.run(
        ["gfortran", "-I", "/content/CAMB/fortran", "-c", str(src_path)],
        check=True
    )
    print("✅ Cell 57: Successfully patched and compiled Interpolation.f90")
except subprocess.CalledProcessError as e:
    print("❌ Cell 57: Compilation failed.")
    print(e)

❌ Cell 57: Compilation failed.
Command '['gfortran', '-I', '/content/CAMB/fortran', '-c', '/content/CAMB/forutils/Interpolation.f90']' returned non-zero exit status 1.
